In [1]:
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.pgvector import PGVector
from langchain.document_loaders import TextLoader
from langchain.docstore.document import Document
from langchain.vectorstores.pgvector import PGVector

import os

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')


In [7]:

loader = TextLoader('./data/turtle_space.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [8]:
print(len(docs))

10


In [9]:
CONNECTION_STRING = PGVector.connection_string_from_db_params(
    driver=os.environ.get("PGVECTOR_DRIVER", "psycopg"),
    host=os.getenv('DB_HOST'),
    port=os.getenv('DB_PORT'),
    database=os.getenv('DB_DATABASE'),
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD')
)

In [10]:
from typing import List, Tuple

# The PGVector Module will try to create a table with the name of the collection. 
# So, make sure that the collection name is unique and the user has the permission to create a table.
db = PGVector.from_documents(
    embedding=embeddings,
    documents=docs,
    collection_name="state_of_the_union",
    connection_string=CONNECTION_STRING,
)


In [11]:

query = "What did the aliens do?"
docs_with_score: List[Tuple[Document, float]] = db.similarity_search_with_score(query)

In [12]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

--------------------------------------------------------------------------------
Score:  0.5400543305807132
The spaceship landed on the beach, and a door opened. A group of aliens stepped out. They were tall and green, with big eyes and long necks. Leonardo was scared at first, but the aliens smiled at him and said hello.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.5723379017028418
The aliens told Leonardo that they were from a planet far away. They had come to Earth to learn about its inhabitants. Leonardo was happy to help them. He told them about his life on Earth, and about his dream of going to space.
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Score:  0.5850036521097922
Before the aliens left, they gave Leonardo a gift. It was a small spa